In [1]:
import pymc as pm
import numpy as np
import arviz as az
import pandas as pd
import pytensor.tensor as at
from pytensor.tensor.subtensor import set_subtensor

%load_ext lab_black

# Wine Classification*

Adapted from [Unit 10: italywines123.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/italywines123.odc).

Data can be found [here](https://archive.ics.uci.edu/ml/datasets/wine).

This popular data set was provided by Forina et al (1988). 
The data below consist of results of a chemical analysis of wines grown in the same region in Italy but derived from three different cultivars. The analysis determined the quantities of 13 constituents found in each of the three types of wines. 

|Column|Variable|Description|
|---|---|---|
|1|Y|Type (Response, 1,2,3)|
|---|---|1 = [1 0 0]; 2 = [0 1 0]; 3 = [0 0 1]|
|2|X1|Alcohol|
|3|X2|Malic acid|
|4|X3|Ash|     
|5|X4|Alkalinity of ash| 
|6|X5|Magnesium|
|7|X6|Total phenols|
|8|X7|Flavanoids|
|9|X8|Nonflavanoid phenols||
|10|X9|Proanthocyanins|
|11|X10|Color intensity|
|12|X11|Hue|
|13|X12|OD280/OD315 of diluted wines|
|14|X13|Proline|

(a) Fit the multinomial regressionthat predicts the type of wine Y from predictors X1 - X13. What are estimated coefficients? What is the deviance?

(b) What is your prediction for pp=P(Ynew=1) if a new case has attributes ```new_attributes``` (below) How would you classify this wine type, as 1, 2, or 3?

Forina, M., Leardi, R., Armanino, C., and Lanteri, S. (1988). PARVUS: An extendable package of programs for data exploration, classification and correlation,  Elsevier, Amsterdam,   ISBN 0-444-43012-1;

Report at Institute of Pharmaceutical and Food Analysis and Technologies, Via Brigata Salerno, 16147 Genoa, Italy.

In [2]:
data = pd.read_csv("../data/wine.data", header=None)
Y = pd.get_dummies(data[0]).to_numpy()
X = data.drop(0, axis=1).to_numpy()
X_aug = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

In [3]:
X_aug.shape, Y.shape

((178, 14), (178, 3))

In [4]:
with pm.Model() as m:
    X_data = pm.Data("X", X_aug, mutable=True)
    Y_data = pm.Data("y", Y, mutable=False)
    _b = pm.Normal("_b", 0, tau=0.05, shape=(14, 2))
    b = at.concatenate([at.zeros((14, 1)), _b], axis=1)

    # 178, 14 x 14, 3 -> 178, 3
    phi = pm.math.exp(pm.math.dot(X_data, b))
    # probabilities must sum to 1
    P = phi / pm.math.sum(phi, axis=1)[:, None]

    # P is 178, 3. category count determined by last axis
    pm.Multinomial("likelihood", n=1, p=P, observed=Y_data)

    trace = pm.sample(2000, init="jitter+adapt_diag_grad", target_accept=0.95)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag_grad...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [_b]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 45 seconds.


In [5]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
"_b[0, 0]",3.962,4.358,-4.010,12.311,0.045,0.039,9351.0,6517.0,1.0
"_b[0, 1]",-0.538,4.428,-8.788,7.925,0.040,0.052,11962.0,5516.0,1.0
"_b[1, 0]",3.126,1.402,0.518,5.740,0.031,0.022,2109.0,3581.0,1.0
"_b[1, 1]",0.233,1.909,-3.338,3.789,0.037,0.026,2690.0,3520.0,1.0
"_b[2, 0]",-3.671,1.670,-6.790,-0.581,0.028,0.021,3577.0,4449.0,1.0
"_b[2, 1]",1.003,2.203,-3.066,5.195,0.032,0.023,4623.0,4893.0,1.0
"_b[3, 0]",-7.831,3.102,-13.615,-2.005,0.050,0.036,3795.0,4789.0,1.0
"_b[3, 1]",0.624,4.016,-7.002,8.031,0.049,0.043,6760.0,5776.0,1.0
"_b[4, 0]",1.522,0.510,0.603,2.489,0.010,0.007,2778.0,3597.0,1.0
"_b[4, 1]",0.913,0.892,-0.730,2.577,0.015,0.011,3525.0,4419.0,1.0


Coefficients are not the same as BUGS results.

In [6]:
new_attributes = np.array(
    [1, 12.9, 2, 2.4, 17, 100, 2.8, 2.1, 0.35, 1.6, 5, 1.05, 3, 750]
).reshape((1, 14))
pm.set_data({"X": new_attributes}, model=m)
ppc = pm.sample_posterior_predictive(trace, model=m)

Sampling: [likelihood]


In [7]:
ppc.posterior_predictive.mean(
    dim=["chain", "draw", "likelihood_dim_2"]
).likelihood.values

array([0.92029354, 0.07548244, 0.00422402])

Getting the same predicted category, but with slightly different probabilities.

In [8]:
%load_ext watermark
%watermark -n -u -v -iv -p pytensor

Last updated: Wed Mar 22 2023

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.9.0

pytensor: 2.10.1

pandas  : 1.5.3
arviz   : 0.15.1
numpy   : 1.24.2
pytensor: 2.10.1
pymc    : 5.1.2

